# 教學課程：開始使用 Gemini API

匯入必要的套件

In [ ]:
# 用於處理文件路徑
import pathlib
# 用於格式化文本
import textwrap
# 導入 google.generativeai 套件並簡化名稱為 genai
import google.generativeai as genai
# 用於顯示格式化的文本和 Markdown
from IPython.display import display, Markdown

將 Gemini API 新增至環境變數中

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

查看可用的 Gemini 模型

In [ ]:
# 遍歷所有可用模型
for m in genai.list_models():
    # 檢查模型是否支持 generateContent 方法
    if 'generateContent' in m.supported_generation_methods:
        # 輸出模型名稱
        print(m.name)

進行文本生成

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("人生的意義是什麼？")
print(response.text)

顯示格式化的 Markdown 文字

In [ ]:
# 定義將文本轉換為 Markdown 格式的函數
def to_markdown(text):
    # 替換符號以適應 Markdown 語法
    text = text.replace('•', '  *')
    # 返回格式化的 Markdown 文本
    return Markdown(
        textwrap.indent(
            text, '> ',
            predicate=lambda _: True
        )
    )
# 顯示格式化的文本
display(to_markdown(response.text))

如果 API 無法傳回結果，查看原因

In [ ]:
print(response.prompt_feedback)

多模態

In [ ]:
# 導入 PIL 用於圖像處理
import PIL.Image
# 打開圖片
img = PIL.Image.open('image.png')

model = genai.GenerativeModel('gemini-1.5-flash')
# 使用多模態輸入生成內容
# 若要指定使用繁體中文，務必在提示中具體指出
response = model.generate_content([
    # 提示文本
    "基於這張圖片，用繁體中文寫一篇有趣且引人入勝的博客。"
    "這張圖片展示了一道美食，我想要一篇中文的描述和背景故事。",
    # 圖片對象
    img
    ],
    # 啟用流式輸出
    stream=True
)
# 遍歷生成的內容塊
for chunk in response:
    print(chunk.text)

即時通訊對話

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash')
# 開始一個新的聊天會話
chat = model.start_chat(history=[])

# 發送訊息取得回答
response = chat.send_message(
    "請用一句話解釋電腦是怎麼運作的，給小孩聽。"
)
# 輸出回答
print(response.text)

# 流式
# 發送新的訊息並啟用流式輸出
for chunk in chat.send_message(
    "那麼給高中生的詳細解釋呢？",
    stream=True
):
    print(chunk.text)

使用嵌入

In [ ]:
# 使用嵌入功能
result = genai.embed_content(
    # 指定模型
    model="models/embedding-001",
    # 提示文本
    content="什麼是人生的意義？",
    # 指定任務類型
    task_type="retrieval_document",
    # 指定標題
    title="單一字串的嵌入"
)
# 輸出嵌入向量的前50個值
print(result['embedding'][:50], '... TRIMMED]')

設定模型安全性參數

In [ ]:
# 使用安全性設定生成內容
response = model.generate_content(
    # 提示文本
    '[疑似有問題的提示]',
    # 設定安全性參數
    safety_settings={'HARASSMENT':'block_none'}
)
print(response.text)

使用 `genai.protos.Content` 類型進行訊息編碼

In [ ]:
response = model.generate_content(
    genai.protos.Content(
        parts=[
            genai.protos.Part(
                text="請基於這張圖片，使用繁體中文寫一篇有趣的博客。"
            ),
            genai.protos.Part(
                inline_data=genai.protos.Blob(
                    mime_type='image/jpeg',
                    data=pathlib.Path('image.png').read_bytes()
                )
            ),
        ]
    ), 
    stream=True
)

response.resolve()
# 之後才可以訪問 response 的屬性
print(response.text[:100], "... [TRIMMED] ...")


多輪對話

使用 `ChatSession` 管理對話

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash')
chat = model.start_chat(history=[])

response = chat.send_message(
    "請使用繁體中文，用一段話簡單介紹 Gemini 是怎麼運作的給大學生理解。"
)
print(response.text)

延續對話並使用流式輸出

In [ ]:
_message = "如果是要解釋給研究生及博士生理解呢？"
for chunk in chat.send_message(_message, stream=True):
    print(chunk.text)

In [ ]:
_message = "如果是要小學生理解呢？"
for chunk in chat.send_message(_message, stream=True):
    print(chunk.text)

延續對話

In [ ]:
# 初始訊息
messages = [{
    'role': 'user',
    'parts': ["請簡短解釋電腦是如何運作的給小學三年級學生聽。"]
}]
# 使用模型生成內容
response = model.generate_content(messages)
# 回覆
print(response.text)

# 添加模型回覆到對話記錄
messages.append({
    'role': 'model',
    'parts': [response.text]
})

# 繼續對話
messages.append({
    'role': 'user',
    'parts': ["接著進一步詳細解釋給大學生聽。"]
})
# 使用模型生成新的內容
response = model.generate_content(messages)
# 回覆
print(response.text)

產生設定

In [ ]:
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content(
    # 提示文本
    '講一個關於神奇背包的故事。',
    generation_config=genai.types.GenerationConfig(
        # 設定生成候選數量
        candidate_count=1,
        # 設定停止序列
        stop_sequences=['x'],
        # 設定最大輸出權杖數
        max_output_tokens=20,
        # 設定溫度參數
        temperature=1.0
    )
)
# 取得生成的文本
text = response.text
# 檢查生成是否達到最大權杖數
if response.candidates[0].finish_reason.name == "MAX_TOKENS":
    # 添加省略號表示未完成的內容
    text += '...'
# 輸出
print(text)

嵌入功能

In [ ]:
result = genai.embed_content(
    model="models/embedding-001",
    content="什麼是人生的意義？",
    task_type="retrieval_document",
    title="單一字串的嵌入"
)
print(result['embedding'][:50], '... TRIMMED]')

處理一批字串的嵌入

In [ ]:
result = genai.embed_content(
    model="models/embedding-001",
    content=[
        '什麼是人生的意義？',
        '一隻木頭做的狗能吃多少木頭？',
        '大腦是如何運作的？'
    ],
    task_type="retrieval_document",
    title="字串清單的嵌入"
)

for v in result['embedding']:
    print(str(v)[:50], '... TRIMMED ...')

安全性設定

In [ ]:
response = model.generate_content(
    '[疑似有問題的提示]',
    safety_settings={'HARASSMENT': 'block_none'}
)
print(response.text)